In [ ]:
import pandas as pd
import geopandas as gpd
from shapely.wkt import loads
import osmnx as ox
import sys
import subprocess

def find_correct_crs():
    """
    주어진 고도/경사도 데이터에 대해, 가능한 좌표계 목록을 테스트하여
    강남구 영역과 일치하는 올바른 좌표계(CRS)를 판별합니다.
    """
    # --- 0. 라이브러리 준비 ---
    try:
        import openpyxl
    except ImportError:
        print("엑셀 파일을 읽기 위해 'openpyxl' 라이브러리를 설치합니다.")
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'openpyxl'])

    # --- 1. 테스트할 좌표계(CRS) 목록 정의 ---
    # 서울/한국에서 사용될 가능성이 높은 좌표계 목록
    candidate_crs_list = [
        'EPSG:5186',  # Korea 2000 / Central Belt 2010 (가장 유력)
        'EPSG:5185',  # Korea 2000 / West Belt 2010
        'EPSG:5187',  # Korea 2000 / East Belt 2010
        'EPSG:5179',  # Korea 1985 / Unified CS (과거에 많이 사용)
        'EPSG:5174',  # Bessel 1975 / Modified Korean Central Belt
        'EPSG:32652', # WGS 84 / UTM Zone 52N
    ]
    print(f"좌표계 판별 시작. 총 {len(candidate_crs_list)}개의 후보를 테스트합니다.")
    print("----------------------------------------------------------")

    # --- 2. 데이터 불러오기 (최초 1회) ---
    try:
        elevation_df = pd.read_excel('data/경사도데이터/서울시_고도_경사도_경사각.xlsx')
        elevation_df['geometry'] = elevation_df['geometry'].apply(loads)
    except FileNotFoundError:
        print("❌ 오류: '서울시_고도_경사도_경사각.xlsx' 파일을 찾을 수 없습니다.")
        return

    place_name = 'Gangnam-gu, Seoul, South Korea'
    area_gdf = ox.geocode_to_gdf(place_name)

    found_crs = None

    # --- 3. 각 후보 좌표계를 순회하며 판별 작업 수행 ---
    for crs_candidate in candidate_crs_list:
        print(f"▶️ 후보 '{crs_candidate}' 테스트 중...")

        try:
            # 3-1. 고도 데이터를 현재 후보 좌표계로 가정
            elevation_gdf = gpd.GeoDataFrame(elevation_df, geometry='geometry', crs=crs_candidate)

            # 3-2. 강남구 영역도 같은 후보 좌표계로 변환
            area_projected = area_gdf.to_crs(elevation_gdf.crs)

            # 3-3. 두 데이터가 겹치는지 확인 (공간 조인)
            elevation_in_gangnam = gpd.sjoin(elevation_gdf, area_projected, how="inner")
            points_in_gangnam = len(elevation_in_gangnam)

            print(f"   ...결과: 강남구 내에서 {points_in_gangnam}개의 일치하는 지점을 찾았습니다.")

            # 3-4. 겹치는 데이터가 있으면, 정답으로 간주하고 종료
            if points_in_gangnam > 0:
                found_crs = crs_candidate
                print(f"\n✅ 정답을 찾았습니다! 이 데이터의 좌표계는 '{found_crs}'일 가능성이 매우 높습니다.")
                break # 정답을 찾았으므로 루프 중단

        except Exception as e:
            # 지원하지 않는 좌표계 변환 등의 오류 발생 시
            print(f"   ...오류: '{crs_candidate}' 처리 중 오류 발생. ({e})")

        print("-" * 25)


    # --- 4. 최종 결론 ---
    print("\n----------------- 최종 판별 결과 -----------------")
    if found_crs:
        print(f"이 엑셀 파일은 '{found_crs}' 좌표계를 사용하고 있습니다.")
        print(f"이제 이 좌표계('{found_crs}')를 사용해서 데이터를 결합하면 됩니다.")
    else:
        print("모든 주요 좌표계를 테스트했지만, 강남구와 일치하는 데이터를 찾지 못했습니다.")
        print("이 파일에는 강남구 데이터가 없거나, 전혀 다른 좌표계를 사용하고 있을 수 있습니다.")
    print("----------------------------------------------------------")

if __name__ == '__main__':
    find_correct_crs()

In [4]:
from pyproj import CRS, Transformer, exceptions

def convert_coords_and_compare(coord_pairs):
    if not coord_pairs:
        print("입력된 좌표가 없습니다.")
        return
    # (X, Y) 쌍으로 묶인 리스트를 X 리스트와 Y 리스트로 분리
    try:
        x_coords, y_coords = zip(*coord_pairs)
    except ValueError:
        print("❌ 오류: 좌표 입력 형식을 확인해주세요. 예: [(x1, y1), (x2, y2)]")
        return
    # --------------------------------------------------

    # 대한민국에서 주로 사용되는 좌표계 후보 목록
    candidate_crs_list = {
        "커스텀 Bessel (Tokyo Datum)": "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-146.43,507.89,681.46",
        "EPSG:5186 (GRS80 / 중부 2010)": "EPSG:5186",
        "EPSG:5185 (GRS80 / 서부 2010)": "EPSG:5185",
        "EPSG:5187 (GRS80 / 동부 2010)": "EPSG:5187",
        "EPSG:5179 (UTM-K)": "EPSG:5179",
        "EPSG:5174 (Bessel / 중부)": "EPSG:5174",
        "EPSG:32652 (WGS84 / UTM Zone 52N)": "EPSG:32652"
    }
    
    # 목표 좌표계는 항상 WGS84 (위도/경도)
    target_crs = CRS("EPSG:4326")

    print("\n--- 🕵️ 여러 좌표계로 변환 테스트 시작 ---")
    print("결과 중 대한민국 영토(위도 33~39, 경도 124~132) 안에 들어오는 것을 찾으세요.\n")

    for name, crs_string in candidate_crs_list.items():
        try:
            source_crs = CRS(crs_string)
            transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)
            lon, lat = transformer.transform(x_coords, y_coords)
            
            print(f"✅ 테스트: {name}")
            print(f"  - 변환 결과 샘플:")
            
            sample_count = min(3, len(lon))
            for i in range(sample_count):
                print(f"    > 원본 {coord_pairs[i]} -> 변환(경도: {lon[i]:.5f}, 위도: {lat[i]:.5f})")
            print("-" * 50)

        except exceptions.ProjError as e:
            print(f"❌ 실패: {name}")
            print(f"  -> 에러: {e}\n")
            print("-" * 50)

# --- 메인 코드 실행 부분 ---
if __name__ == '__main__':
    # 여기에 (X, Y) 쌍으로 묶어서 좌표를 넣어봐!
    sample_coords = [
        (336658.07, 608782.41),  # 형이 말한 좌표
        (339681.62, 600045.47),  # 이전 예시 좌표 1
        (265994.63, 539220.69)   # 이전 예시 좌표 2
    ]

    convert_coords_and_compare(sample_coords)


--- 🕵️ 여러 좌표계로 변환 테스트 시작 ---
결과 중 대한민국 영토(위도 33~39, 경도 124~132) 안에 들어오는 것을 찾으세요.

✅ 테스트: 커스텀 Bessel (Tokyo Datum)
  - 변환 결과 샘플:
    > 원본 (336658.07, 608782.41) -> 변환(경도: 127.27575, 위도: 38.07960)
    > 원본 (339681.62, 600045.47) -> 변환(경도: 127.31095, 위도: 38.00109)
    > 원본 (265994.63, 539220.69) -> 변환(경도: 126.48333, 위도: 37.44532)
--------------------------------------------------
✅ 테스트: EPSG:5186 (GRS80 / 중부 2010)
  - 변환 결과 샘플:
    > 원본 (336658.07, 608782.41) -> 변환(경도: 128.55730, 위도: 38.06880)
    > 원본 (339681.62, 600045.47) -> 변환(경도: 128.59005, 위도: 37.98966)
    > 원본 (265994.63, 539220.69) -> 변환(경도: 127.74583, 위도: 37.45004)
--------------------------------------------------
✅ 테스트: EPSG:5185 (GRS80 / 서부 2010)
  - 변환 결과 샘플:
    > 원본 (336658.07, 608782.41) -> 변환(경도: 126.55730, 위도: 38.06880)
    > 원본 (339681.62, 600045.47) -> 변환(경도: 126.59005, 위도: 37.98966)
    > 원본 (265994.63, 539220.69) -> 변환(경도: 125.74583, 위도: 37.45004)
--------------------------------------------------
✅ 테스트: EPSG:5187 

### 필수 코드로 넣을거

In [ ]:
from pyproj import CRS, Transformer, exceptions

# --- 메인 코드 실행 부분 ---
sample_coords = [
    (336658.07, 608782.41),  # 좌표 1
    (339681.62, 600045.47),  # 좌표 2
    (265994.63, 539220.69)   # 좌표 3
]

coord_pairs = sample_coords

if not coord_pairs:
    print("입력된 좌표가 없습니다.")
    raise SystemExit
# (X, Y) 쌍으로 묶인 리스트를 X 리스트와 Y 리스트로 분리
try:
    x_coords, y_coords = zip(*coord_pairs)
except ValueError:
    print("오류: 좌표 입력 형식을 확인해주세요. 예: [(x1, y1), (x2, y2)]")
    raise SystemExit
# --------------------------------------------------

# 대한민국에서 주로 사용되는 좌표계 후보 목록
candidate_crs_list = {
    "커스텀 Bessel (Tokyo Datum)": "+proj=tmerc +lat_0=38 +lon_0=128 +k=0.9999 +x_0=400000 +y_0=600000 +ellps=bessel +units=m +no_defs +towgs84=-146.43,507.89,681.46",
    "EPSG:5186 (GRS80 / 중부 2010)": "EPSG:5186",
    "EPSG:5185 (GRS80 / 서부 2010)": "EPSG:5185",
    "EPSG:5187 (GRS80 / 동부 2010)": "EPSG:5187",
    "EPSG:5179 (UTM-K)": "EPSG:5179",
    "EPSG:5174 (Bessel / 중부)": "EPSG:5174",
    "EPSG:32652 (WGS84 / UTM Zone 52N)": "EPSG:32652"
}

# 목표 좌표계는 항상 WGS84 (위도/경도)
target_crs = CRS("EPSG:4326")

print("\n--- 여러 좌표계로 변환 테스트 시작 ---")
print("대한민국 영토(위도 33~39, 경도 124~132)\n")

for name, crs_string in candidate_crs_list.items():
    try:
        source_crs = CRS(crs_string)
        transformer = Transformer.from_crs(source_crs, target_crs, always_xy=True)
        lon, lat = transformer.transform(x_coords, y_coords)
        
        print(f" 테스트: {name}")
        print(f"  - 변환 결과 샘플:")
        
        sample_count = min(3, len(lon))
        for i in range(sample_count):
            print(f"    > 원본 {coord_pairs[i]} -> 변환(경도: {lon[i]:.5f}, 위도: {lat[i]:.5f})")
        print("-" * 50)

    except exceptions.ProjError as e:
        print(f" 실패: {name}")
        print(f"  -> 에러: {e}\n")
        print("-" * 50)